In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import tensorflow as tf
from tqdm import tqdm
from glob import glob
from google.colab import drive

import librosa
import librosa.display as dsp
import IPython.display as ipd

warnings.filterwarnings(action='ignore')
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/deep-voice/data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/deep-voice/data


In [ ]:
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') #GPU 할당

In [ ]:
import random

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(813)

In [ ]:
# 저장된 데이터 불러오기
X_train_mfcc = np.load('X_train_mfcc.npy')
X_val_mfcc = np.load('X_val_mfcc.npy')
y_train = np.load('y_train.npy')
y_val = np.load('y_val.npy')

## LSTM

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# MFCC 등의 데이터를 위한 LSTM 모델 정의
# input_shape=(시간 단계 수, 특성 수)

model = Sequential([
    LSTM(50, input_shape=(100, 2970), return_sequences=True),
    LSTM(50),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 요약 출력
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 100, 50)           604200    
                                                                 
 lstm_4 (LSTM)               (None, 50)                20200     
                                                                 
 dense_2 (Dense)             (None, 10)                510       
                                                                 
Total params: 624910 (2.38 MB)
Trainable params: 624910 (2.38 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

# 모델 훈련
history = model.fit(
    X_train_mfcc, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_mfcc, y_val),
    callbacks=[early_stopping]
)

Epoch 1/10
40/40 [==============================] - 21s 432ms/step - loss: 0.5180 - accuracy: 0.9391 - val_loss: 0.0470 - val_accuracy: 0.9937
Epoch 2/10
40/40 [==============================] - 16s 398ms/step - loss: 0.0116 - accuracy: 0.9992 - val_loss: 0.0083 - val_accuracy: 1.0000
Epoch 3/10
40/40 [==============================] - 15s 367ms/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 0.0044 - val_accuracy: 1.0000
Epoch 4/10
40/40 [==============================] - 16s 406ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.0036 - val_accuracy: 1.0000
Epoch 5/10
40/40 [==============================] - 17s 429ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.0029 - val_accuracy: 1.0000
Epoch 6/10
40/40 [==============================] - 16s 402ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0026 - val_accuracy: 1.0000
Epoch 7/10
40/40 [==============================] - 15s 371ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.0023 - val_accuracy: 1.0000

## CNN-LSTM

In [ ]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dropout, Dense, Input
from tensorflow.keras.models import Sequential
from tensorflow_addons.layers import GroupNormalization

model = Sequential()

# Conv1D 레이어 추가. input_shape의 마지막 차원을 2970으로 설정
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(100, 2970)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# LSTM 레이어 추가
model.add(LSTM(50))

# 분류를 위한 완전 연결 레이어 및 출력 레이어 추가
model.add(Dense(10, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(
    monitor='val_loss',  # 관찰할 값
    patience=3,          # 성능 향상이 없는 에폭을 몇 번이나 허용할 것인가
    verbose=1,           # 진행 상황을 출력
    mode='min',          # 'min' 모드는 관찰하는 값의 감소가 멈추면 훈련을 중단
    restore_best_weights=True  # 가장 좋은 모델의 가중치를 복원
)

# 모델 훈련
history = model.fit(
    X_train_mfcc, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_mfcc, y_val),
    callbacks=[early_stopping]  # 콜백 리스트에 early_stopping 추가
)

Epoch 1/10
40/40 [==============================] - 14s 306ms/step - loss: 0.2653 - accuracy: 0.9406 - val_loss: 0.0111 - val_accuracy: 0.9969
Epoch 2/10
40/40 [==============================] - 11s 274ms/step - loss: 0.0056 - accuracy: 1.0000 - val_loss: 0.0065 - val_accuracy: 1.0000
Epoch 3/10
40/40 [==============================] - 11s 278ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.0045 - val_accuracy: 1.0000
Epoch 4/10
40/40 [==============================] - 9s 234ms/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0037 - val_accuracy: 1.0000
Epoch 5/10
40/40 [==============================] - 11s 284ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0033 - val_accuracy: 1.0000
Epoch 6/10
40/40 [==============================] - 11s 282ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0027 - val_accuracy: 1.0000
Epoch 7/10
40/40 [==============================] - 11s 272ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.0023 - val_accuracy: 1.0000


## Bi-LSTM

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dropout, Dense
from tensorflow_addons.layers import GroupNormalization

model = Sequential()

model.add(Bidirectional(LSTM(512, input_shape=(X_train_mfcc.shape[1], 1))))
model.add(GroupNormalization())
model.add(Dropout(0.8))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'], run_eagerly=True)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(
    monitor='val_loss',  # 관찰할 값
    patience=3,          # 성능 향상이 없는 에폭을 몇 번이나 허용할 것인가
    verbose=1,           # 진행 상황을 출력
    mode='min',          # 'min' 모드는 관찰하는 값의 감소가 멈추면 훈련을 중단
    restore_best_weights=True  # 가장 좋은 모델의 가중치를 복원
)

# 모델 훈련
history = model.fit(
    X_train_mfcc, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_mfcc, y_val),
    callbacks=[early_stopping]  # 콜백 리스트에 early_stopping 추가
)

Epoch 1/10
40/40 [==============================] - 390s 10s/step - loss: 0.1210 - accuracy: 0.9578 - val_loss: 0.0161 - val_accuracy: 0.9937
Epoch 2/10
40/40 [==============================] - 376s 9s/step - loss: 7.0005e-04 - accuracy: 1.0000 - val_loss: 0.0227 - val_accuracy: 0.9937
Epoch 3/10
40/40 [==============================] - 378s 9s/step - loss: 8.4539e-04 - accuracy: 1.0000 - val_loss: 0.0216 - val_accuracy: 0.9937
Epoch 4/10
40/40 [==============================] - 373s 9s/step - loss: 0.0014 - accuracy: 0.9992 - val_loss: 0.0089 - val_accuracy: 0.9969
Epoch 5/10
40/40 [==============================] - 375s 9s/step - loss: 3.0970e-04 - accuracy: 1.0000 - val_loss: 0.0112 - val_accuracy: 0.9969
Epoch 6/10
40/40 [==============================] - 373s 9s/step - loss: 2.2241e-04 - accuracy: 1.0000 - val_loss: 0.0101 - val_accuracy: 0.9969
Epoch 7/10
40/40 [==============================] - 374s 9s/step - loss: 1.6721e-04 - accuracy: 1.0000 - val_loss: 0.0117 - val_accuracy:

## CNN-BiLSTM

In [ ]:
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 13.8 MB/s eta 0:00:00


In [ ]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dropout, Dense, Input
from tensorflow.keras.models import Sequential
from tensorflow_addons.layers import GroupNormalization

model = Sequential()

# Conv1D 레이어 추가. input_shape의 마지막 차원을 2970으로 설정
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(100, 2970)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
# LSTM 레이어 전에 데이터의 차원을 맞추기 위해 필요한 경우 Flatten 레이어 또는 GlobalMaxPooling1D 레이어를 추가할 수 있습니다.
# 이 예시에서는 데이터가 시퀀스이므로, Flatten 대신 Bidirectional LSTM을 직접 사용합니다.
model.add(Bidirectional(LSTM(512)))
model.add(GroupNormalization())
model.add(Dropout(0.8))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'], run_eagerly=True)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(
    monitor='val_loss',  # 관찰할 값
    patience=3,          # 성능 향상이 없는 에폭을 몇 번이나 허용할 것인가
    verbose=1,           # 진행 상황을 출력
    mode='min',          # 'min' 모드는 관찰하는 값의 감소가 멈추면 훈련을 중단
    restore_best_weights=True  # 가장 좋은 모델의 가중치를 복원
)

# 모델 훈련
history = model.fit(
    X_train_mfcc, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_mfcc, y_val),
    callbacks=[early_stopping]  # 콜백 리스트에 early_stopping 추가
)

Epoch 1/10
40/40 [==============================] - 57s 1s/step - loss: 0.1352 - accuracy: 0.9477 - val_loss: 0.0015 - val_accuracy: 1.0000
Epoch 2/10
40/40 [==============================] - 53s 1s/step - loss: 3.4366e-04 - accuracy: 1.0000 - val_loss: 0.0028 - val_accuracy: 0.9969
Epoch 3/10
40/40 [==============================] - 55s 1s/step - loss: 2.4034e-04 - accuracy: 1.0000 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 4/10
40/40 [==============================] - 55s 1s/step - loss: 6.9047e-04 - accuracy: 1.0000 - val_loss: 0.0065 - val_accuracy: 0.9969
Epoch 4: early stopping


## RNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# RNN 모델 정의
model = Sequential([
    SimpleRNN(50, input_shape=(100, 2970), return_sequences=True),
    SimpleRNN(50),
    # 10개의 출력 클래스에 대한 Dense 레이어. 분류를 위해 softmax 활성화 함수를 사용합니다.
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 모델 요약
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 100, 50)           151050    
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 50)                5050      
                                                                 
 dense (Dense)               (None, 10)                510       
                                                                 
Total params: 156610 (611.76 KB)
Trainable params: 156610 (611.76 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(
    monitor='val_loss',  # 관찰할 값
    patience=3,          # 성능 향상이 없는 에폭을 몇 번이나 허용할 것인가
    verbose=1,           # 진행 상황을 출력
    mode='min',          # 'min' 모드는 관찰하는 값의 감소가 멈추면 훈련을 중단
    restore_best_weights=True  # 가장 좋은 모델의 가중치를 복원
)

# 모델 훈련
history = model.fit(
    X_train_mfcc, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_mfcc, y_val),
    callbacks=[early_stopping]  # 콜백 리스트에 early_stopping 추가
)

Epoch 1/10
40/40 [==============================] - 12s 242ms/step - loss: 0.6819 - accuracy: 0.8102 - val_loss: 0.1663 - val_accuracy: 0.9750
Epoch 2/10
40/40 [==============================] - 10s 247ms/step - loss: 0.0432 - accuracy: 0.9977 - val_loss: 0.0649 - val_accuracy: 0.9906
Epoch 3/10
40/40 [==============================] - 7s 177ms/step - loss: 0.0145 - accuracy: 1.0000 - val_loss: 0.0451 - val_accuracy: 0.9937
Epoch 4/10
40/40 [==============================] - 9s 217ms/step - loss: 0.0087 - accuracy: 1.0000 - val_loss: 0.0381 - val_accuracy: 0.9937
Epoch 5/10
40/40 [==============================] - 11s 272ms/step - loss: 0.0061 - accuracy: 1.0000 - val_loss: 0.0330 - val_accuracy: 0.9937
Epoch 6/10
40/40 [==============================] - 7s 177ms/step - loss: 0.0046 - accuracy: 1.0000 - val_loss: 0.0296 - val_accuracy: 0.9937
Epoch 7/10
40/40 [==============================] - 9s 230ms/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.0270 - val_accuracy: 0.9937
Epo

## GRU

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

# 모델 정의
model = Sequential([
    GRU(50, input_shape=(100, 2970), return_sequences=True),  # LSTM 대신 GRU 사용
    GRU(50),  # 여기도 동일하게 GRU 사용
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 요약
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 100, 50)           453300    
                                                                 
 gru_1 (GRU)                 (None, 50)                15300     
                                                                 
 dense_3 (Dense)             (None, 10)                510       
                                                                 
Total params: 469110 (1.79 MB)
Trainable params: 469110 (1.79 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(
    monitor='val_loss',  # 관찰할 값
    patience=3,          # 성능 향상이 없는 에폭을 몇 번이나 허용할 것인가
    verbose=1,           # 진행 상황을 출력
    mode='min',          # 'min' 모드는 관찰하는 값의 감소가 멈추면 훈련을 중단
    restore_best_weights=True  # 가장 좋은 모델의 가중치를 복원
)

# 모델 훈련
history = model.fit(
    X_train_mfcc, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_mfcc, y_val),
    callbacks=[early_stopping]  # 콜백 리스트에 early_stopping 추가
)

Epoch 1/10
40/40 [==============================] - 20s 408ms/step - loss: 0.4989 - accuracy: 0.8867 - val_loss: 0.0284 - val_accuracy: 0.9969
Epoch 2/10
40/40 [==============================] - 13s 326ms/step - loss: 0.0073 - accuracy: 1.0000 - val_loss: 0.0252 - val_accuracy: 0.9969
Epoch 3/10
40/40 [==============================] - 14s 346ms/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 0.0232 - val_accuracy: 0.9969
Epoch 4/10
40/40 [==============================] - 14s 341ms/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 0.0225 - val_accuracy: 0.9969
Epoch 5/10
40/40 [==============================] - 13s 326ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.0221 - val_accuracy: 0.9969
Epoch 6/10
40/40 [==============================] - 13s 320ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0220 - val_accuracy: 0.9969
Epoch 7/10
40/40 [==============================] - 14s 351ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.0220 - val_accuracy: 0.9969

## Bi-GRU

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, GRU, Dense

# 모델 정의
model = Sequential([
    Bidirectional(GRU(50, return_sequences=True), input_shape=(100, 2970)),  # 첫 번째 GRU 레이어를 양방향으로 설정
    Bidirectional(GRU(50)),  # 두 번째 GRU 레이어도 양방향으로 설정
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 요약
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 100, 100)          906600    
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 100)               45600     
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 953210 (3.64 MB)
Trainable params: 953210 (3.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(
    monitor='val_loss',  # 관찰할 값
    patience=3,          # 성능 향상이 없는 에폭을 몇 번이나 허용할 것인가
    verbose=1,           # 진행 상황을 출력
    mode='min',          # 'min' 모드는 관찰하는 값의 감소가 멈추면 훈련을 중단
    restore_best_weights=True  # 가장 좋은 모델의 가중치를 복원
)

# 모델 훈련
history = model.fit(
    X_train_mfcc, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_mfcc, y_val),
    callbacks=[early_stopping]  # 콜백 리스트에 early_stopping 추가
)

Epoch 1/10
40/40 [==============================] - 19s 383ms/step - loss: 0.5548 - accuracy: 0.8672 - val_loss: 0.0310 - val_accuracy: 0.9969
Epoch 2/10
40/40 [==============================] - 13s 335ms/step - loss: 0.0083 - accuracy: 1.0000 - val_loss: 0.0227 - val_accuracy: 0.9969
Epoch 3/10
40/40 [==============================] - 14s 341ms/step - loss: 0.0041 - accuracy: 1.0000 - val_loss: 0.0213 - val_accuracy: 0.9969
Epoch 4/10
40/40 [==============================] - 13s 337ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.0208 - val_accuracy: 0.9969
Epoch 5/10
40/40 [==============================] - 13s 331ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.0207 - val_accuracy: 0.9969
Epoch 6/10
40/40 [==============================] - 13s 328ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.0208 - val_accuracy: 0.9969
Epoch 7/10
40/40 [==============================] - 14s 338ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0209 - val_accuracy: 0.9969